In [ ]:
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# set cwd back to default
os.chdir('../')
os.getcwd()

In [ ]:
## NN libs
import keras
from sklearn.decomposition import PCA
from keras.utils import to_categorical
from keras.layers import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Conv3D, MaxPool2D, Dropout, Flatten
from keras.callbacks import TensorBoard

In [ ]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

# Prepare dataset

In [ ]:
import data # src/data.py
dataset = data.init_dataset()

### Select the amount of classes that will be used

In [ ]:
# pick the n classes with the most occuring instances
amt = 3
classes = data.top_classes(dataset.labels, amt)
classes

In [ ]:
name_list = []
n_per_class = []
tail = '.jpg'
for cls in classes:
    names = data.items_with_label(dataset.labels, cls)
    train_names = [f for f in names if (f + tail) in dataset.train]
    name_list.append(train_names)
    n_per_class.append(len(train_names))
    
n = min(n_per_class)
# (optional) reduce n to check whether the model can rember its input
reduced_n = 50
if n > reduced_n:    n = reduced_n
x = []
for ls in name_list:
    for name in ls[:n]:
        x.append(name)
random.shuffle(x)
len(x)

In [ ]:
# TODO rmv faces

## Load & convert images
All input images should have the same size

In [ ]:
# TODO use crop
# TODO first 
x_train, y_train, n = data.extract_all(dataset, x)
n

In [ ]:
data.show_info(x_train)

In [ ]:
plot.multiple(x_train[:10])

### Prepare the labels

Encode the labels to one-hot vectors

In [ ]:
# TODO y_test
y_test = y_train
y_train, y_test = data.labels_to_vectors(dataset, y_train, y_test)

In [ ]:
y_train[0]

# Train a Sequential model (keras)

In [ ]:
n_samples = x_train.shape[0] # = length of the list of images (matrices)
input_shape = x_train.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label
print(n_samples, input_shape)
print('output length', output_length)

In [ ]:
x_train.shape

In [ ]:
input_shape

In [ ]:
def sequential_conv(input_shape, output_length, dropout=0.10):
    # Convolutional layers
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2))) # strides=(2,2) - watch dims!?!
    model.add(Conv2D(32, (9, 9), activation='relu'))
    # reduce the number of dimensions
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(8, (6, 6), activation='relu'))
    model.add(Dropout(dropout))  
    # Dense layers
    model.add(Flatten())  
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(output_length,activation='softmax'))
    return model, model.summary

dropout = 0.10
model, summary = sequential_conv(input_shape, output_length, dropout)
summary()

In [ ]:
# model, summary = models.sequential_conv(input_shape, output_length)
# summary()

## Loss function
- Categorical cross-entropy loss

In [ ]:
learning_rate = 0.001
# Adam, SGD
# sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
optimizer = Adam(lr=learning_rate)
# top_k_categorical_accuracy(y_true, y_pred, k=5)
# https://keras.io/metrics/
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',
    'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Train the model

In [ ]:
batch_size = 8
# n epochs = n iterations over all the training data
epochs = 10

In [ ]:
config.tmp_model_dir

In [ ]:
# model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=1/6)
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

In [ ]:
result = model.predict(x_train[:10])
# result

In [ ]:
result[0]

i = result[0].argmax()
result[0][i]

In [ ]:
np.mean(np.array(result[0]))